# Database Engineering

All database engineering will be completed within this notebook. 
SQLAlchemy will be utilized to model table schemas and create a sqlite database for the created tables.

In [185]:
# Import SQLAlchemy and other dependencies 
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
import pandas as pd

In [186]:
#create an engine
engine = create_engine("sqlite:///hawaii.sqlite")

# Create a connection to the engine called `conn`
conn = engine.connect()

### Creating tables

In [187]:
# Use `declarative_base` from SQLAlchemy to model the measurements table as an ORM class
# Make sure to specify types for each column, e.g. Integer, Text, etc.
# http://docs.sqlalchemy.org/en/latest/core/type_basics.html
Base = declarative_base()


# Create Measurement Class
# ----------------------------------
class Measurements(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(15))
    date = Column(String(15))
    prcp = Column(Float)
    tobs = Column(Integer)

    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [188]:
# Create Station class
# ---------------------------------------
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(15))
    name = Column(String(45))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [189]:
# Use `create_all` to create the measurements table in the database
Base.metadata.create_all(engine)

In [190]:
engine.table_names()

['measurements', 'station']

In [191]:
# Load the cleaned csv file into a pandas dataframe
df_of_measurements = pd.read_csv('clean_hawaii_measurements.csv')
df_of_stations = pd.read_csv('clean_hawaii_stations.csv')

In [192]:
# Use Orient='records' to create a list of data to write
station_data = df_of_stations.to_dict(orient='records')

In [193]:
weather_data = df_of_measurements.to_dict(orient='records')
weather_data[0]

{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}

In [194]:
# Use MetaData from SQLAlchemy to reflect the tables

metadata = MetaData(bind=engine)
metadata.reflect()

In [195]:
# Save the reference to the `demographics` table as a variable called `table`

weather_table = sqlalchemy.Table('measurements', metadata, autoload=True)
station_table = sqlalchemy.Table('station', metadata, autoload=True)

In [196]:
# Use `table.delete()` to remove any existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.

conn.execute(weather_table.delete())

In [197]:
conn.execute(station_table.delete())

In [198]:
# Use `table.insert()` to insert the data into the table

conn.execute(weather_table.insert(), weather_data)
conn.execute(station_table.insert(), station_data)

In [199]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [200]:
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]